In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchtext import data, datasets
from matplotlib import pyplot as plt
import numpy as np
from tqdm.notebook import tqdm

dev = 'cuda' if torch.cuda.is_available() else 'cpu'

In [2]:
tokenize_en = data.get_tokenizer("spacy", language='en_core_web_sm')
tokenize_de = data.get_tokenizer("spacy", language='de_core_news_sm')

src = data.Field(tokenize_en)
tgt = data.Field(tokenize_de)

train, val, test = datasets.Multi30k.splits(
    ('.en', '.de'), fields=(src, tgt) , root='./downloads')

train.fields['src'].build_vocab(train)
train.fields['trg'].build_vocab(train)

In [3]:
def collate_fn(batch):
    src_list, trg_list = [], []
    for dt_pnt in batch:
        src_list.append(dt_pnt.src)
        trg_list.append(dt_pnt.trg)

    src_list = train.fields['src'].pad(src_list)
    trg_list = train.fields['trg'].pad(trg_list)

    src_list = train.fields['src'].numericalize(src_list)
    trg_list = train.fields['trg'].numericalize(trg_list)

    return src_list, trg_list


dl = DataLoader(
    train, shuffle=True, batch_size=16, collate_fn=collate_fn)
next(iter(dl))[0].shape

torch.Size([29, 16])